In [28]:
import numpy as np

In [29]:
# Newmark's method
def newmark(K, M, C, u0, v0, acel0, p, n_steps, dt, gamma, beta):
    #acel0 = np.matmul(np.linalg.inv(M),(p0 - C*v0 - K*u0))

    # Degrees of freedom
    dofs = K.shape[0]
    # Integration constants
    alpha = [
        1.0/(beta*dt**2.0),
        gamma/(beta*dt),
        1.0/(beta*dt),
        1.0/(2.0*beta)-1.0,
        gamma/beta-1.0,
        (gamma/beta-2.0)*dt/2.0,
        (1.0-gamma)*dt,
        gamma*dt
        ]

    # Effective stiffness matrix
    Keff = K + alpha[0]*M + alpha[1]*C
    # print(Keff)
    # Keff inverse
    Kinv = np.linalg.inv(Keff)
    # print(Kinv)

    # Integration
    # Vector of effective forces at time p
    pef = np.zeros((n_steps+1,dofs))
    # Vector of displacements at time u
    u = np.zeros((n_steps+1,dofs))
    u[0] = u0
    # Vector of accelerations  and velocities at time
    acel = np.zeros((n_steps+1,dofs))
    acel[0] = acel0
    v = np.zeros((n_steps+1,dofs))
    v[0] = v0
    for n in range(n_steps):
        # Vector of effective forces p
        term1 = alpha[0]*u[n] + alpha[2]*v[n] + alpha[3]*acel[n]
        term2 = alpha[1]*u[n] + alpha[4]*v[n] + alpha[5]*acel[n]
        pef[n+1] = p[n+1] + np.matmul(M,term1) + np.matmul(C,term2)
        # Vector of displacements u
        u[n+1] = np.matmul(Kinv,pef[n+1])
        # Vector of accelerations acel and velocities v 
        acel[n+1] = alpha[0]*(u[n+1]-u[n]) - alpha[2]*v[n] - alpha[3]*acel[n]
        v[n+1] = v[n] + alpha[6]*acel[n] + alpha[7]*acel[n+1]
    # print(pef)  
    
    return u,v,acel

In [30]:
# Example 9.2 Paultre book (page 244)

M_t = np.array([[5.0*10.0**5.0]])  # kg
K_t = np.array([[3.*10.0**8.0]])  # N/m
C_t = np.array([[1224744.9]])  # Ns/m
dofs = K_t.shape[0]
u0_t = np.array([0.0])
v0_t = np.array([0.0])
acel0 = np.array([0.0])
n_steps = 10
dt_t = 0.05
gamma_t = 1.0/2.0
beta_t = 1.0/4.0

# Function p(t): in this example p is a half cycle sine function  
p_t = np.zeros((n_steps+1, dofs))
n_points = n_steps + 1
l = np.linspace(0, 0.5, n_points)
p_t = np.array([[5000*np.sin(2*np.pi*x) for x in l]])
p_t = np.round(p_t,8)
p_t = np.transpose(p_t)

# Newmark's integration
u,v,acel = newmark(K_t, M_t, C_t, u0_t, v0_t, acel0, p_t, n_steps, dt_t, gamma_t, beta_t)
print('u = ')
print(u)
print('v = ')
print(v)
print('acel = ')
print(acel)

u = 
[[0.00000000e+00]
 [1.34473341e-06]
 [6.41766328e-06]
 [1.43363825e-05]
 [2.03445947e-05]
 [2.07421864e-05]
 [1.64234487e-05]
 [1.14699446e-05]
 [8.69730195e-06]
 [7.05863157e-06]
 [3.43499541e-06]]
v = 
[[ 0.00000000e+00]
 [ 5.37893366e-05]
 [ 1.49127858e-04]
 [ 1.67620911e-04]
 [ 7.27075764e-05]
 [-5.68039077e-05]
 [-1.15945601e-04]
 [-8.21945641e-05]
 [-2.87111403e-05]
 [-3.68356750e-05]
 [-1.08109771e-04]]
acel = 
[[ 0.00000000e+00]
 [ 2.15157346e-03]
 [ 1.66196739e-03]
 [-9.22245268e-04]
 [-2.87428811e-03]
 [-2.30617125e-03]
 [-5.94964823e-05]
 [ 1.40953795e-03]
 [ 7.29798999e-04]
 [-1.05478039e-03]
 [-1.79618346e-03]]


In [31]:
# Example 22.2 Paultre book (page 683)
M = 20.0*10.0**3 * np.array([[1.0,0.0],[0.0,1.0]])
K = 18.0*10.0**6 * np.array([[2.0,-1.0],[-1.0,1.0]])
C = np.array([[0.0,0.0],[0.0,0.0]])
u0 = np.array([[0.02,0.02]])
v0 = np.array([[0.0,0.0]])
acel0 = np.array([[-18.0,0.0]])
n_steps = 10
p = np.zeros((n_steps+1,dofs))
dt = 0.01
gamma = 1.0/2.0
beta = 1.0/4.0
u,v,acel = newmark(K, M, C, u0, v0, acel0, p, n_steps, dt, gamma, beta)
print('u = ')
print(u)
print('v = ')
print(v)
print('acel = ')
print(acel)

u = 
[[ 0.02        0.02      ]
 [ 0.01913835  0.01998104]
 [ 0.01669865  0.01985318]
 [ 0.01308594  0.01942185]
 [ 0.00887959  0.01841977]
 [ 0.00471208  0.01657883]
 [ 0.00113855  0.01370644]
 [-0.00147399  0.00974899]
 [-0.00301378  0.0048273 ]
 [-0.00363313 -0.00076431]
 [-0.00369767 -0.00659622]]
v = 
[[ 0.          0.        ]
 [-0.17233045 -0.00379211]
 [-0.31560943 -0.02177962]
 [-0.40693305 -0.06448663]
 [-0.43433552 -0.13592904]
 [-0.3991669  -0.23226019]
 [-0.31553889 -0.34221605]
 [-0.20697046 -0.44927499]
 [-0.10098718 -0.53506327]
 [-0.02288154 -0.58325782]
 [ 0.00997324 -0.58312403]]
acel = 
[[-18.           0.        ]
 [-16.46609044  -0.75842253]
 [-12.18970434  -2.83907918]
 [ -6.07502018  -5.70232219]
 [  0.5945264   -8.58616023]
 [  6.43919665 -10.68006959]
 [ 10.28640571 -11.31110258]
 [ 11.4272802  -10.10068499]
 [  9.76937676  -7.05697218]
 [  5.8517503   -2.5819365 ]
 [  0.71920667   2.60869447]]


In [32]:
# Newmark's method for each time step (use inside a for loop on n_steps)
def newmark_step(K, M, C, u, v, acel, p_next, dt, gamma, beta):
    #acel0 = np.matmul(np.linalg.inv(M),(p0 - C*v0 - K*u0))

    # Degrees of freedom
    dofs = K.shape[0]
    # Integration constants
    alpha = [
        1.0/(beta*dt**2.0),
        gamma/(beta*dt),
        1.0/(beta*dt),
        1.0/(2.0*beta)-1.0,
        gamma/beta-1.0,
        (gamma/beta-2.0)*dt/2.0,
        (1.0-gamma)*dt,
        gamma*dt
        ]

    # Effective stiffness matrix
    Keff = K + alpha[0]*M + alpha[1]*C
    
    # Keff inverse
    Kinv = np.linalg.inv(Keff)
    

    # Integration
    # Vector of effective forces at time
    pef = np.zeros((dofs))
    # Vector of displacements at time
    u_next = np.zeros((dofs))
    # Vector of accelerations  and velocities at time
    acel_next = np.zeros((dofs))
    v_next = np.zeros((dofs))

    # Vector of effective forces p
    term1 = alpha[0]*u + alpha[2]*v + alpha[3]*acel
    term2 = alpha[1]*u + alpha[4]*v + alpha[5]*acel
    pef_next = p_next + np.matmul(M,term1) + np.matmul(C,term2)
    # Vector of displacements u
    u_next = np.matmul(Kinv,pef_next)
    # Vector of accelerations acel and velocities v 
    acel_next = alpha[0]*(u_next-u) - alpha[2]*v - alpha[3]*acel
    v_next = v + alpha[6]*acel + alpha[7]*acel_next  
    
    return u_next,v_next,acel_next


In [33]:
# Example 9.2 Paultre book (page 244)

M = np.array([[5.0*10.0**5.0]])  # kg
K = np.array([[3.*10.0**8.0]])  # N/m
C = np.array([[1224744.9]])  # Ns/m
dofs = K_t.shape[0]
u0 = np.array([0.0])
v0 = np.array([0.0])
acel0 = np.array([0.0])
u = u0
v = v0
acel = acel0
n_steps = 10
dt = 0.05
gamma = 1.0/2.0
beta = 1.0/4.0

# Function p(t): in this example p is a half cycle sine function  
p = np.zeros((n_steps+1, dofs))
n_points = n_steps + 1
l = np.linspace(0, 0.5, n_points)
p = np.array([[5000*np.sin(2*np.pi*x) for x in l]])
p = np.round(p,8)
p = np.transpose(p)

for n in range(n_steps):
    u,v,acel = newmark_step(K,M,C,u,v,acel,p[n+1],dt,gamma,beta)
    print('step = ')
    print(n+1)
    print('u = ')
    print(u)
    # print('v = ')
    # print(v)
    # print('acel = ')
    # print(acel)

step = 
1
u = 
[1.34473341e-06]
step = 
2
u = 
[6.41766328e-06]
step = 
3
u = 
[1.43363825e-05]
step = 
4
u = 
[2.03445947e-05]
step = 
5
u = 
[2.07421864e-05]
step = 
6
u = 
[1.64234487e-05]
step = 
7
u = 
[1.14699446e-05]
step = 
8
u = 
[8.69730195e-06]
step = 
9
u = 
[7.05863157e-06]
step = 
10
u = 
[3.43499541e-06]


In [34]:

# Example 22.2 Paultre book (page 683)
M = 20.0*10.0**3 * np.array([[1.0,0.0],[0.0,1.0]])
K = 18.0*10.0**6 * np.array([[2.0,-1.0],[-1.0,1.0]])
C = np.array([[0.0,0.0],[0.0,0.0]])
u0 = np.array([0.02,0.02])
v0 = np.array([0.0,0.0])
acel0 = np.array([-18.0,0.0])
u = u0
v = v0
acel = acel0
n_steps = 10
dofs = K.shape[0]
p = np.zeros((n_steps+1,dofs))
dt = 0.01
gamma = 1.0/2.0
beta = 1.0/4.0

for n in range(n_steps):
    u,v,acel = newmark_step(K,M,C,u,v,acel,p[n+1],dt,gamma,beta)
    print('step = ')
    print(n)
    print('u = ')
    print(u)
    # print('v = ')
    # print(v)
    # print('acel = ')
    # print(acel)

step = 
0
u = 
[0.01913835 0.01998104]
step = 
1
u = 
[0.01669865 0.01985318]
step = 
2
u = 
[0.01308594 0.01942185]
step = 
3
u = 
[0.00887959 0.01841977]
step = 
4
u = 
[0.00471208 0.01657883]
step = 
5
u = 
[0.00113855 0.01370644]
step = 
6
u = 
[-0.00147399  0.00974899]
step = 
7
u = 
[-0.00301378  0.0048273 ]
step = 
8
u = 
[-0.00363313 -0.00076431]
step = 
9
u = 
[-0.00369767 -0.00659622]
